### Actividad:  repaso de threading, multiprocessing, asyncio, concurrent.futures

#### Ejercicio 1: Uso básico de threading

Crea un script que inicie 5 hilos. Cada hilo debe imprimir un mensaje que indique su número de identificación (ID) y luego esperar un tiempo aleatorio entre 1 y 5 segundos antes de finalizar.

In [ ]:
import threading
import time
import random

def worker_thread(thread_id):
    # Cada hilo imprime su ID.
    print(f"El hilo {thread_id} esta corriendo.")
    
    # Genera un tiempo de espera aleatorio entre 1 y 5 segundos.
    sleep_time = random.randint(1, 5)
    time.sleep(sleep_time)  # Corrección aquí: eliminación del argumento con nombre
    
    # Mensaje de finalización del hilo.
    print(f"Hilos {thread_id} a finalizado despues de {sleep_time} segundos.")

# Lista para mantener los hilos.
threads = []

# Crear y lanzar 5 hilos.
for i in range(5):
    # Creamos el hilo.
    thread = threading.Thread(target=worker_thread, args=(i,))
    
    # Añadimos el hilo a la lista de hilos.
    threads.append(thread)
    
    # Iniciar el hilo.
    thread.start()

# Esperar a que todos los hilos terminen.
for thread in threads:
    thread.join()

print("Todos los hilos es finalizado.")


#### Ejercicio 2: Uso de multiprocessing

Escribe un programa que utilice multiprocessing para calcular el factorial de diferentes números de forma paralela. Usa los números 5, 7, y 9.

In [ ]:
import multiprocessing

def factorial(number):
    if number == 0 or number == 1:
        return 1
    else:
        result = 1
        for i in range(2, number + 1):
            result *= i
        return result

def compute_factorial(number, result_dict):
    # Calcular el factorial y almacenarlo en un diccionario compartido.
    result_dict[number] = factorial(number)
    print(f"El factorial de {number} es {result_dict[number]}")

if __name__ == '__main__':
    # Lista de números para los cuales queremos calcular el factorial.
    numbers = [5, 7, 9]
    
    # Diccionario que se usa para almacenar los resultados.
    # Utilizamos un Manager dict para permitir el acceso entre procesos.
    manager = multiprocessing.Manager()
    result_dict = manager.dict()
    
    # Lista para mantener los procesos.
    processes = []
    
    # Crear y lanzar un proceso por cada número.
    for number in numbers:
        # Creamos el proceso.
        process = multiprocessing.Process(target=compute_factorial, args=(number, result_dict))
        
        # Añadimos el proceso a la lista de procesos.
        processes.append(process)
        
        # Iniciar el proceso.
        process.start()
    
    # Esperar a que todos los procesos terminen.
    for process in processes:
        process.join()
    
    print("Todos los procesos terminados.")


#### Ejercicio 3: Uso de asyncio

Utiliza asyncio para simular la descarga de tres archivos de manera asíncrona. Los tiempos de "descarga" deben ser de 3, 5 y 7 segundos respectivamente.

In [ ]:
import asyncio

async def download_file(file_name, duration):
    # Simula la descarga esperando la cantidad de segundos especificada.
    print(f"Iniciando la descarga de {file_name} (durará {duration} segundos)...")
    await asyncio.sleep(duration)
    print(f"Descarga completada: {file, duration} segundos")

async def main():
    # Crear las "tareas" para cada descarga simulada.
    await asyncio.gather(
        download_file("archivo1.txt", 3),
        download_file("archivo2.txt", 5),
        download_file("archivo3.txt", 7)
    )

# Ejecutar la función main del programa.
asyncio.run(main())


#### Ejercicio 4: Uso de concurrent.futures con ThreadPoolExecutor

Escribe un programa que utilice un ThreadPoolExecutor para realizar un cálculo simple (como la suma de números) en varios hilos. Utiliza un rango de números del 1 al 10.

In [ ]:
import concurrent.futures

def sum_range(start, end):
    total = 0
    for number in range(start, end + 1):
        total += number
    return total

def main():
    ranges = [(1, 3), (4, 6), (7, 10)]  # Dividiendo el rango de números en tres partes.
    results = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=3) as executor:
        # Crear futuros para cada rango de sumas.
        futures = [executor.submit(sum_range, r[0], r[1]) for r in ranges]
        
        # Recopilar los resultados a medida que se completan.
        for future in concurrent.futures.as_completed(futures):
            result = future.result()
            print(f"Resultado obtenido: {result}")
            results.append(result)
    
    # Sumar los resultados parciales para obtener el total final.
    total_sum = sum(results)
    print(f"La suma total de 1 a 10 es: {total_sum}")

if __name__ == "__main__":
    main()


#### Ejercicio 5: Uso de asyncio con corutinas

Implementa un programa con asyncio que simule un reloj, mostrando cada segundo un mensaje que indica cuántos segundos han pasado, hasta un total de 10 segundos.



In [ ]:
import asyncio

async def clock():
    for second in range(1, 11):  # Iterar de 1 a 10 segundos
        print(f"{second} segundo{'s' if second > 1 else ''} han pasado")
        await asyncio.sleep(1)  # Espera un segundo

async def main():
    await clock()  # Ejecutar la corutina del reloj

# Ejecutar la función main utilizando asyncio
asyncio.run(main())



Observación: El error que se experimenta sugiere que se está intentando ejecutar asyncio.run(main()) dentro de un entorno que ya tiene un bucle de eventos en ejecución. Esto es común cuando se utiliza asyncio en un entorno interactivo como Jupyter Notebook o IPython.

Para manejar esta situación, puedes modificar la forma en que ejecutas tu programa para trabajar con el bucle de eventos ya existente.

In [ ]:
import asyncio

async def clock():
    for second in range(1, 11):  # Iterar de 1 a 10 segundos
        print(f"{second} segundo{'s' if second > 1 else ''} han pasado")
        await asyncio.sleep(1)  # Espera un segundo

# Código ajustado para un entorno con bucle de eventos en ejecución
async def main():
    await clock()  # Ejecutar la corutina del reloj

# Utilizar get_event_loop para obtener el bucle de eventos actual
loop = asyncio.get_event_loop()

# Verificar si el bucle de eventos está en ejecución
if loop.is_running():
    # Ejecutar la corutina en el bucle de eventos ya existente
    asyncio.ensure_future(main())
else:
    # Iniciar el bucle de eventos y correr la corutina
    loop.run_until_complete(main())


#### Ejercicio 6: Uso de multiprocessing con Pool

Utiliza un Pool de procesos en multiprocessing para aplicar una función que calcule el cuadrado de un número a una lista de números del 1 al 10 de manera paralela.

In [ ]:
import multiprocessing

def square_number(n):
    return n * n

def main():
    numbers = range(1, 11)  # Lista de números del 1 al 10
    with multiprocessing.Pool() as pool:
        # map aplica la función 'square_number' a cada elemento de la lista 'numbers'
        results = pool.map(square_number, numbers)
        print(results)

if __name__ == '__main__':
    # Correr la función main si el script es ejecutado como principal
    main()


#### Ejercicio 7: Programación funcional con map y reduce

Implementa un script que use map para convertir todas las palabras en una lista a mayúsculas, y luego use reduce para concatenarlas en una sola cadena separada por comas.

In [ ]:
def main():
    words = ['hola', 'mundo', 'programación', 'funcional', 'Python']
    # Convertir todas las palabras a mayúsculas
    upper_words = map(str.upper, words)
    
    # Concatenar todas las palabras en una cadena separada por comas usando un bucle for
    result = ""
    for word in upper_words:
        if result:  # Si result ya tiene contenido, añadimos una coma antes del siguiente elemento
            result += ","
        result += word

    print(result)

if __name__ == '__main__':
    main()


#### Ejercicio 8: Crear un microservicio simple con Flask

Desarrolla un microservicio básico usando Flask que tenga un endpoint /api/square que acepte un número y devuelva su cuadrado.

In [ ]:
%%witefile server.py

from flask import Flask, request, jsonify

app = Flask(__name__)

@app.route('/api/square', methods=['GET'])
def square():
    # Obtener el parámetro 'number' de la URL
    number = request.args.get('number', default=1, type=int)
    
    # Calcular el cuadrado del número
    result = number ** 2
    
    # Devolver el resultado en formato JSON
    return jsonify({'numero': number, 'cuadrado': result})

if __name__ == '__main__':
    app.run(debug=True, port=5001) # Cambia el puerto si es necesario.


Esto iniciará el servidor Flask, y podrás acceder a él normalmente desde http://localhost:5001.

Para probar el endpoint /api/square, puedes usar un navegador o una herramienta como curl para hacer una solicitud GET. Por ejemplo, para calcular el cuadrado de 5:

In [ ]:
curl "http://localhost:5001/api/square?number=5"

Esto debería devolver una respuesta en JSON como:

In [ ]:
{
  "numero": 5,
  "cuadrado": 25
}


#### Ejercicio 9: Uso de asyncio y aiohttp para hacer peticiones HTTP concurrentes

Escribe un script que use asyncio y aiohttp para hacer peticiones a tres URLs diferentes de manera concurrente y recolecte sus resultados.

In [ ]:
import asyncio
import aiohttp

async def fetch(session, url):
    """Petición HTTP asincrónica a la URL dada."""
    async with session.get(url) as response:
        return await response.json()  # Analizar la respuesta JSON

async def main():
    urls = [
        'https://jsonplaceholder.typicode.com/todos/1',
        'https://api.open-meteo.com/v1/forecast?latitude=35.6895&longitude=139.6917&current_weather=true',
        'https://httpbin.org/delay/2'  # Este endpoint introduce un retraso de 2 segundos
    ]
    # Crear una sesión aiohttp
    async with aiohttp.ClientSession() as session:
        tasks = [fetch(session, url) for url in urls]
        # Esperar todas las respuestas
        responses = await asyncio.gather(*tasks)
        for response in responses:
            print(response)  # Imprimir cada respuesta JSON

# Ejecutar el bucle de eventos
asyncio.run(main())



Observación: El error indica que ya hay un bucle de eventos de asyncio ejecutándose y estás intentando iniciar uno nuevo con asyncio.run(). Este problema es común cuando intentas ejecutar código asyncio dentro de un entorno como Jupyter Notebook o en el contexto de algunas aplicaciones de servidor que ya manejan su propio bucle de eventos.

Para resolver este problema en un entorno como Jupyter Notebook, en lugar de usar asyncio.run(), debes utilizar las funciones get_event_loop() para obtener el bucle de eventos existente y luego run_until_complete() para ejecutar tu corutina principal.

In [ ]:
%%writefile asyncio1.py  # utiliza este archivo
import asyncio
import aiohttp

async def fetch(session, url):
    async with session.get(url) as response:
        return await response.json()  # Parse JSON response

async def main():
    urls = [
        'https://jsonplaceholder.typicode.com/todos/1',
        'https://api.open-meteo.com/v1/forecast?latitude=35.6895&longitude=139.6917&current_weather=true',
        'https://httpbin.org/delay/2'  # This endpoint introduces a delay of 2 seconds
    ]
    async with aiohttp.ClientSession() as session:
        tasks = [fetch(session, url) for url in urls]
        responses = await asyncio.gather(*tasks)
        for response in responses:
            print(response)  

loop = asyncio.get_event_loop()
loop.run_until_complete(main())


Para solucionar esto en un entorno como Jupyter, puedes utilizar la librería nest_asyncio, que permite anidar bucles de eventos de asyncio.

pip install nest_asyncio




Una vez instalado, puedes aplicar nest_asyncio a tu bucle de eventos existente para permitir que se ejecute correctamente en entornos como Jupyter Notebook. 

In [2]:
import asyncio
import aiohttp
import nest_asyncio

# Aplicar nest_asyncio
nest_asyncio.apply()

async def fetch(session, url):
    async with session.get(url) as response:
        return await response.json() 

async def main():
    urls = [
        'https://jsonplaceholder.typicode.com/todos/1',
        'https://api.open-meteo.com/v1/forecast?latitude=35.6895&longitude=139.6917&current_weather=true',
        'https://httpbin.org/delay/2' 
    ]
    async with aiohttp.ClientSession() as session:
        tasks = [fetch(session, url) for url in urls]
        responses = await asyncio.gather(*tasks)
        for response in responses:
            print(response)  
asyncio.run(main())


{'userId': 1, 'id': 1, 'title': 'delectus aut autem', 'completed': False}
{'latitude': 35.7, 'longitude': 139.6875, 'generationtime_ms': 0.0400543212890625, 'utc_offset_seconds': 0, 'timezone': 'GMT', 'timezone_abbreviation': 'GMT', 'elevation': 40.0, 'current_weather_units': {'time': 'iso8601', 'interval': 'seconds', 'temperature': '°C', 'windspeed': 'km/h', 'winddirection': '°', 'is_day': '', 'weathercode': 'wmo code'}, 'current_weather': {'time': '2024-05-11T18:00', 'interval': 900, 'temperature': 16.4, 'windspeed': 6.5, 'winddirection': 186, 'is_day': 0, 'weathercode': 1}}
{'args': {}, 'data': '', 'files': {}, 'form': {}, 'headers': {'Accept': '*/*', 'Accept-Encoding': 'gzip, deflate', 'Host': 'httpbin.org', 'User-Agent': 'Python/3.8 aiohttp/3.8.1', 'X-Amzn-Trace-Id': 'Root=1-663fb59d-03579ec82115402911b7d30c'}, 'origin': '190.238.29.202', 'url': 'https://httpbin.org/delay/2'}


Este enfoque permite ejecutar código asyncio en Jupyter Notebook sin conflictos con el bucle de eventos del entorno.

#### Ejercicio 10: Uso de concurrent.futures para procesamiento distribuido

Implementa un script que use concurrent.futures y ThreadPoolExecutor para calcular el resultado de aplicar una función compleja (como el cálculo de la serie Fibonacci) a diferentes entradas de manera distribuida.

In [2]:
import concurrent.futures

def fibonacci(n):
    """ Calcula el n-ésimo número de Fibonacci. """
    if n <= 1:
        return n
    else:
        return fibonacci(n-1) + fibonacci(n-2)

def main():
    # Lista de números para los cuales calcular la serie Fibonacci
    nums = [10, 20, 30, 35, 40]
    
    # Usar ThreadPoolExecutor para calcular los números de Fibonacci de forma concurrente
    with concurrent.futures.ThreadPoolExecutor() as executor:
        # Usamos map para aplicar la función a cada elemento en nums
        results = executor.map(fibonacci, nums)
        
        # Imprimir resultados
        for num, result in zip(nums, results):
            print(f'Fibonacci({num}) = {result}')

if __name__ == "__main__":
    main()


Fibonacci(10) = 55
Fibonacci(20) = 6765
Fibonacci(30) = 832040
Fibonacci(35) = 9227465
Fibonacci(40) = 102334155


#### Ejercicio 11: Uso de asyncio con manejo de excepciones

Desarrolla un script utilizando asyncio que realice peticiones HTTP a varios endpoints. Utiliza manejo de excepciones para tratar adecuadamente los errores de conexión y otros errores HTTP.

In [5]:
import aiohttp
import asyncio

async def fetch_url(session, url):
    try:
        async with session.get(url) as response:
            if response.status == 200:
                data = await response.text()
                print(f"Respuesta exitosa de {url}: {data[:100]}...")  # Imprime los primeros 100 caracteres
            else:
                print(f"Error {response.status} desde {url}")
    except aiohttp.ClientConnectionError:
        print(f"Error de conexión con {url}")
    except aiohttp.ClientError as e:
        print(f"Error al realizar la petición a {url}: {e}")

async def main(urls):
    async with aiohttp.ClientSession() as session:
        tasks = [fetch_url(session, url) for url in urls]
        await asyncio.gather(*tasks)

# Ejecuta directamente la función main con await en Jupyter
if __name__ == "__main__":
    urls = [
        "http://example.com",
        "http://nonexistent.url",
        "http://httpbin.org/status/404",
        "http://httpbin.org/status/500"
    ]
    
    # Directamente usar await aquí si es una celda de Jupyter
    await main(urls)


Error de conexión con http://nonexistent.url
Respuesta exitosa de http://example.com: <!doctype html>
<html>
<head>
    <title>Example Domain</title>

    <meta charset="utf-8" />
    <m...
Error 500 desde http://httpbin.org/status/500
Error 404 desde http://httpbin.org/status/404


#### Ejercicio 12: Multiprocessing con uso de Queue para intercambio de datos

Implementa un programa que use multiprocessing donde varios procesos productores generan números aleatorios y un consumidor los suma. Usa Queue para la comunicación entre procesos.

In [7]:
import multiprocessing
import random
import time

def producer(queue):
    """ Función que simula un productor de números aleatorios. """
    for _ in range(10):
        num = random.randint(1, 100)
        queue.put(num)
        print(f"Producido {num}")
        time.sleep(random.random())
    # Indicar que la producción ha terminado
    queue.put(None)

def consumer(queue, num_producers):
    """ Función que simula un consumidor que suma los números recibidos. """
    total_sum = 0
    termination_count = 0
    
    while True:
        num = queue.get()
        if num is None:
            termination_count += 1
            if termination_count == num_producers:
                break
        else:
            total_sum += num
            print(f"Consumido {num}, la suma total es ahora {total_sum}")
    
    print(f"La suma final es {total_sum}")

def main():
    # Número de procesos productores
    num_producers = 3
    
    # Crear una cola para comunicar números entre productores y consumidor
    queue = multiprocessing.Queue()
    
    # Crear y comenzar los procesos productores
    producers = [multiprocessing.Process(target=producer, args=(queue,)) for _ in range(num_producers)]
    for p in producers:
        p.start()
    
    # Crear y comenzar el proceso consumidor
    consumer_process = multiprocessing.Process(target=consumer, args=(queue, num_producers))
    consumer_process.start()
    
    # Esperar a que todos los procesos productores terminen
    for p in producers:
        p.join()
    
    # Esperar a que el proceso consumidor termine
    consumer_process.join()

if __name__ == "__main__":
    main()


Producido 58
Producido 4
Producido 69
Consumido 58, la suma total es ahora 58
Consumido 4, la suma total es ahora 62
Consumido 69, la suma total es ahora 131
Producido 93Consumido 93, la suma total es ahora 224

Producido 2Consumido 2, la suma total es ahora 226

Producido 86Consumido 86, la suma total es ahora 312

Producido 85Consumido 85, la suma total es ahora 397

Producido 62Consumido 62, la suma total es ahora 459

Producido 20Consumido 20, la suma total es ahora 479

Producido 89Consumido 89, la suma total es ahora 568

Producido 66Consumido 66, la suma total es ahora 634

Producido 72Consumido 72, la suma total es ahora 706

Producido 14Consumido 14, la suma total es ahora 720

Producido 6Consumido 6, la suma total es ahora 726

Producido 55Consumido 55, la suma total es ahora 781

Producido 88Consumido 88, la suma total es ahora 869

Producido 14Consumido 14, la suma total es ahora 883

Producido 49Consumido 49, la suma total es ahora 932

Producido 27Consumido 27, la suma to

#### Ejercicio 13: Microservicios con comunicación entre servicios

Crea dos microservicios con Flask. El primero genera un número aleatorio. El segundo servicio recibe ese número, lo eleva al cuadrado y devuelve el resultado. Implementa la comunicación entre ambos servicios.

Para crear dos microservicios que se comuniquen entre sí usando Flask, podemos definir dos aplicaciones Flask separadas. El primer servicio generará un número aleatorio y el segundo recibirá este número, lo elevará al cuadrado y devolverá el resultado. Vamos a utilizar requests en el segundo servicio para realizar la petición al primero y obtener el número aleatorio.

Antes de empezar, asegúrate de tener instalado Flask y Requests. Si no los tienes, puedes instalarlos usando pip:

pip install flask requests

Servicio 1: Generador de números aleatorios


In [ ]:
%%writefile service1.py

from flask import Flask
import random

app = Flask(__name__)

@app.route('/random')
def random_number():
    """Genera un número aleatorio entre 1 y 100."""
    number = random.randint(1, 100)
    return {'number': number}

if __name__ == '__main__':
    app.run(port=5000) ## Cambia de puerto


Este servicio corre en el puerto 5000 y tiene un endpoint /random que devuelve un número aleatorio en formato JSON.

Servicio 2: Calculador del cuadrado

Guarda este código en un archivo llamado service2.py:

In [ ]:
%%writefile service2.py
from flask import Flask, jsonify, request
import requests

app = Flask(__name__)

@app.route('/square', methods=['GET'])
def square():
    """Obtiene un número aleatorio del servicio 1 y devuelve su cuadrado."""
    response = requests.get('http://localhost:5000/random')
    number = response.json()['number']
    squared_number = number ** 2
    return jsonify(squared_number=squared_number)

if __name__ == '__main__':
    app.run(port=5001)


Para probar la comunicación, puedes usar una herramienta como Postman o simplemente un navegador web o curl para acceder a http://localhost:5001/square. Esto debería darte el cuadrado de un número aleatorio generado por el primer servicio.

#### Ejercicio 14: Uso de concurrent.futures para scraping de datos

Implementa un programa que utilice concurrent.futures para hacer scraping de datos de varias páginas web de manera concurrente y recolecte datos específicos.



Para implementar un programa que utiliza concurrent.futures para hacer scraping de datos de varias páginas web de manera concurrente, necesitaremos usar la biblioteca requests para hacer las peticiones HTTP y BeautifulSoup de bs4 para parsear el contenido HTML y extraer los datos necesarios. Primero, asegúrate de tener instaladas ambas bibliotecas, si no, puedes instalarlas con pip:

pip install requests beautifulsoup4


In [8]:
import concurrent.futures
import requests
from bs4 import BeautifulSoup

def fetch_and_parse(url):
    """Función que obtiene y parsea el contenido de una URL dada."""
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()  # Asegura que la respuesta fue exitosa
        soup = BeautifulSoup(response.text, 'html.parser')
        title = soup.find('h1')  # Buscar el primer <h1> tag, que a menudo es el título principal
        if title:
            return (url, title.text.strip())
        else:
            return (url, 'No title found')
    except requests.RequestException as e:
        return (url, f'Error fetching the page: {e}')

def main(urls):
    """Función principal que ejecuta tareas concurrentes."""
    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = list(executor.map(fetch_and_parse, urls))
        
        for url, title in results:
            print(f'URL: {url}\nTitle: {title}\n')

if __name__ == "__main__":
    urls = [
        "https://www.bbc.com",
        "https://www.cnn.com",
        "https://www.nytimes.com",
        "https://www.theguardian.com",
        "https://www.nbcnews.com"
    ]
    main(urls)

URL: https://www.bbc.com
Title: No title found

URL: https://www.cnn.com
Title: No title found

URL: https://www.nytimes.com
Title: New York Times - Top Stories

URL: https://www.theguardian.com
Title: No title found

URL: https://www.nbcnews.com
Title: No title found



/home/c-lara/anaconda3/lib/python3.8/site-packages/bs4/builder/__init__.py:314: RuntimeWarning: coroutine 'main' was never awaited
  for attr in list(attrs.keys()):


#### Ejercicio 15: Sistema de procesamiento de comandos paralelo

Implementa un sistema que utilice concurrent.futures para ejecutar comandos del sistema operativo en paralelo, recolectar sus salidas, y procesar los resultados de forma concurrente.


In [9]:
import concurrent.futures
import subprocess

def execute_command(command):
    """Ejecuta un comando del sistema y retorna la salida."""
    try:
        # Ejecutar el comando y capturar la salida estándar y el error estándar
        result = subprocess.run(command, shell=True, text=True, capture_output=True)
        return (command, result.stdout, result.returncode)
    except subprocess.CalledProcessError as e:
        # En caso de error en la ejecución del comando
        return (command, e.output, e.returncode)

def main(commands):
    """Función principal para ejecutar comandos en paralelo."""
    with concurrent.futures.ThreadPoolExecutor() as executor:
        # Mapa de los comandos a la función execute_command
        results = list(executor.map(execute_command, commands))

        # Procesar los resultados
        for command, output, returncode in results:
            if returncode == 0:
                print(f"Command: {command}\nOutput: {output}\n")
            else:
                print(f"Command: {command}\nError: {output}\nReturn Code: {returncode}\n")

if __name__ == "__main__":
    # Lista de comandos del sistema para ejecutar
    commands = [
        'echo Hello, World!',
        'ls -la',
        'pwd',
        'invalid_command'
    ]
    
    main(commands)


Command: echo Hello, World!
Output: Hello, World!


Command: ls -la
Output: total 452
drwxrwxr-x 11 c-lara c-lara   4096 may 11 21:10 .
drwxr----- 73 c-lara c-lara   4096 may 11 20:31 ..
drwxrwxr-x  2 c-lara c-lara   4096 abr 20 13:06 Actividad0
drwxrwxr-x  2 c-lara c-lara   4096 abr 20 13:06 Actividad2
drwxrwxr-x  7 c-lara c-lara   4096 may  1 18:19 Actividad4
-rw-rw-r--  1 c-lara c-lara  53949 may  6 23:12 Actividad5-C8286.ipynb
drwxrwxr-x  2 c-lara c-lara   4096 may 11 21:07 Actividad6
-rw-rw-r--  1 c-lara c-lara  40168 may 11 21:10 Actividad6-C8286.ipynb
-rw-rw-r--  1 c-lara c-lara  65393 abr 20 13:06 Caso-Refactorizacion.ipynb
-rw-rw-r--  1 c-lara c-lara  19869 may  6 12:19 Concurrencia.ipynb
drwxrwxr-x  2 c-lara c-lara   4096 abr 20 13:06 Evaluacion2
-rw-rw-r--  1 c-lara c-lara  53880 may  6 12:14 Evaluacion5-C8286.ipynb
drwxrwxr-x  2 c-lara c-lara   4096 may  3 13:42 Evaluacion6
-rw-rw-r--  1 c-lara c-lara 113496 may  6 16:00 Evaluacion7-C8286.ipynb
drwxrwxr-x  8 c-lara c-lara  

#### Ejercicio 16: Plataforma de análisis de datos distribuidos

Crea una plataforma de análisis de datos que utilice multiprocessing y concurrent.futures para procesar grandes volúmenes de datos distribuidos entre varios procesos, realizando cálculos estadísticos y generación de reportes.

In [10]:
import concurrent.futures
import multiprocessing
import pandas as pd
import numpy as np

def analyze_data(data_chunk):
    """Función que realiza cálculos estadísticos en un fragmento de datos."""
    result = {
        'mean': np.mean(data_chunk),
        'std_dev': np.std(data_chunk),
        'max': np.max(data_chunk),
        'min': np.min(data_chunk)
    }
    return result

def data_distributor(data, num_workers):
    """Distribuye los datos en fragmentos para cada trabajador."""
    chunk_size = len(data) // num_workers
    return (data[i * chunk_size:(i + 1) * chunk_size] for i in range(num_workers))

def main():
    # Simulando algunos datos grandes
    data = np.random.randn(10000)  # 10,000 puntos de datos aleatorios
    
    num_workers = multiprocessing.cpu_count()  # Número de procesos a utilizar
    
    with concurrent.futures.ProcessPoolExecutor(max_workers=num_workers) as executor:
        # Distribuir datos
        chunks = data_distributor(data, num_workers)
        
        # Ejecutar análisis en paralelo
        results = list(executor.map(analyze_data, chunks))
        
        # Procesar y combinar resultados
        final_report = pd.DataFrame(results)
        print("Reporte final:")
        print(final_report.describe())  # Genera un reporte estadístico del resumen de resultados

if __name__ == "__main__":
    main()


Final Report:
           mean   std_dev       max       min
count  8.000000  8.000000  8.000000  8.000000
mean   0.005696  0.997463  3.576763 -3.215886
std    0.036469  0.011213  0.554748  0.433780
min   -0.055656  0.978360  2.857251 -3.901841
25%   -0.018022  0.991786  3.135587 -3.398615
50%    0.014096  0.997752  3.649276 -3.066676
75%    0.032760  1.001337  3.967593 -2.916686
max    0.047480  1.014811  4.247125 -2.769657


#### Ejercicio 17:Descarga de imágenes en paralelo con concurrent.futures

Escribe un script que utilice concurrent.futures para descargar imágenes en paralelo. Debes descargar al menos cinco imágenes de URLs especificadas y guardarlas en el disco localmente.


In [11]:
import os
import requests
from concurrent.futures import ThreadPoolExecutor

# Lista de URLs de las imágenes que deseamos descargar
image_urls = [
    "https://upload.wikimedia.org/wikipedia/commons/thumb/2/23/Lake_mapourika_NZ.jpeg/320px-Lake_mapourika_NZ.jpeg",
    "https://upload.wikimedia.org/wikipedia/commons/thumb/a/a3/June_odd-eyed-cat.jpg/220px-June_odd-eyed-cat.jpg",
    "https://upload.wikimedia.org/wikipedia/commons/thumb/c/c8/Hawthorne_Bridge%2C_Portland%2C_Oregon_%282018%29_-_2.jpg/320px-Hawthorne_Bridge%2C_Portland%2C_Oregon_%282018%29_-_2.jpg",
    "https://upload.wikimedia.org/wikipedia/commons/thumb/d/d4/NYC_Midtown_Skyline_at_night_-_Jan_2006_edit1.jpg/320px-NYC_Midtown_Skyline_at_night_-_Jan_2006_edit1.jpg",
    "https://upload.wikimedia.org/wikipedia/commons/thumb/4/44/Tulip_-_floriade_canberra.jpg/320px-Tulip_-_floriade_canberra.jpg"
]


def download_image(url):
    """Función para descargar una imagen y guardarla localmente."""
    try:
        response = requests.get(url)
        response.raise_for_status()  # Asegura que la respuesta fue exitosa

        # Extraer nombre de archivo de la URL
        file_name = os.path.basename(url)
        
        # Guardar el archivo en el directorio actual
        with open(file_name, 'wb') as f:
            f.write(response.content)
        print(f"Descarga completada: {file_name}")
    except requests.RequestException as e:
        print(f"Error al descargar {url}: {e}")

def main():
    # Usar un ThreadPoolExecutor para descargar imágenes en paralelo
    with ThreadPoolExecutor(max_workers=5) as executor:
        executor.map(download_image, image_urls)

if __name__ == "__main__":
    main()


Error al descargar https://upload.wikimedia.org/wikipedia/commons/thumb/c/c8/Hawthorne_Bridge%2C_Portland%2C_Oregon_%282018%29_-_2.jpg/320px-Hawthorne_Bridge%2C_Portland%2C_Oregon_%282018%29_-_2.jpg: 404 Client Error: Not Found for url: https://upload.wikimedia.org/wikipedia/commons/thumb/c/c8/Hawthorne_Bridge%2C_Portland%2C_Oregon_%282018%29_-_2.jpg/320px-Hawthorne_Bridge%2C_Portland%2C_Oregon_%282018%29_-_2.jpg
Error al descargar https://upload.wikimedia.org/wikipedia/commons/thumb/d/d4/NYC_Midtown_Skyline_at_night_-_Jan_2006_edit1.jpg/320px-NYC_Midtown_Skyline_at_night_-_Jan_2006_edit1.jpg: 404 Client Error: Not Found for url: https://upload.wikimedia.org/wikipedia/commons/thumb/d/d4/NYC_Midtown_Skyline_at_night_-_Jan_2006_edit1.jpg/320px-NYC_Midtown_Skyline_at_night_-_Jan_2006_edit1.jpg
Descarga completada: 220px-June_odd-eyed-cat.jpg
Error al descargar https://upload.wikimedia.org/wikipedia/commons/thumb/4/44/Tulip_-_floriade_canberra.jpg/320px-Tulip_-_floriade_canberra.jpg: 404 C

#### Ejercicio 18: Asincronía con asyncio

Escribe un script que utilice asyncio para realizar peticiones HTTP asincrónicas a tres APIs diferentes y recolecte sus respuestas.



In [15]:
import aiohttp
import asyncio

async def fetch(session, url):
    """Función asincrónica para realizar una petición HTTP."""
    async with session.get(url) as response:
        return await response.text()

async def main():
    """Función principal que coordina las peticiones HTTP asincrónicas."""
    urls = [
        'https://api.github.com',  # GitHub API para metadatos
        'https://api.ipify.org?format=json',  # API para obtener la IP pública
        'https://jsonplaceholder.typicode.com/todos/1'  # API de ejemplo de JSONPlaceholder
    ]

    async with aiohttp.ClientSession() as session:
        tasks = [fetch(session, url) for url in urls]
        responses = await asyncio.gather(*tasks)

        for response in responses:
            print(response)

# Ejecutar el event loop
if __name__ == '__main__':
    asyncio.run(main())


RuntimeError: asyncio.run() cannot be called from a running event loop

Modificamos el codigo:

In [16]:
import aiohttp
import asyncio

async def fetch(session, url):
    """Función asincrónica para realizar una petición HTTP."""
    async with session.get(url) as response:
        return await response.text()

async def main():
    """Función principal que coordina las peticiones HTTP asincrónicas."""
    urls = [
        'https://api.github.com',  # GitHub API para metadatos
        'https://api.ipify.org?format=json',  # API para obtener la IP pública
        'https://jsonplaceholder.typicode.com/todos/1'  # API de ejemplo de JSONPlaceholder
    ]

    async with aiohttp.ClientSession() as session:
        tasks = [fetch(session, url) for url in urls]
        responses = await asyncio.gather(*tasks)

        for response in responses:
            print(response)

# Obtener el bucle de eventos existente y ejecutar la función principal
loop = asyncio.get_event_loop()
try:
    loop.run_until_complete(main())
except RuntimeError:
    asyncio.run(main())  # Solo para casos donde el primer método falla (como ejecución fuera de notebook)


RuntimeError: asyncio.run() cannot be called from a running event loop

Usamos nest_asyncio

In [19]:
import aiohttp
import asyncio
import nest_asyncio

nest_asyncio.apply()  # Aplica parche a asyncio para permitir anidamiento de bucles

async def fetch(session, url):
    """Función asincrónica para realizar una petición HTTP."""
    async with session.get(url) as response:
        return await response.text()

async def main():
    """Función principal que coordina las peticiones HTTP asincrónicas."""
    urls = [
        'https://api.github.com',  # GitHub API para metadatos
        'https://api.ipify.org?format=json',  # API para obtener la IP pública
        'https://jsonplaceholder.typicode.com/todos/1'  # API de ejemplo de JSONPlaceholder
    ]

    async with aiohttp.ClientSession() as session:
        tasks = [fetch(session, url) for url in urls]
        responses = await asyncio.gather(*tasks)

        for response in responses:
            print(response)

# Ejecutar el script en un entorno con un bucle de eventos ya en ejecución
asyncio.run(main())


{"current_user_url":"https://api.github.com/user","current_user_authorizations_html_url":"https://github.com/settings/connections/applications{/client_id}","authorizations_url":"https://api.github.com/authorizations","code_search_url":"https://api.github.com/search/code?q={query}{&page,per_page,sort,order}","commit_search_url":"https://api.github.com/search/commits?q={query}{&page,per_page,sort,order}","emails_url":"https://api.github.com/user/emails","emojis_url":"https://api.github.com/emojis","events_url":"https://api.github.com/events","feeds_url":"https://api.github.com/feeds","followers_url":"https://api.github.com/user/followers","following_url":"https://api.github.com/user/following{/target}","gists_url":"https://api.github.com/gists{/gist_id}","hub_url":"https://api.github.com/hub","issue_search_url":"https://api.github.com/search/issues?q={query}{&page,per_page,sort,order}","issues_url":"https://api.github.com/issues","keys_url":"https://api.github.com/user/keys","label_searc

#### Ejercicio 19: Uso de corutinas con generadores

Implementa una corutina que genere números primos secuencialmente cada vez que se le envíe un valor. Usa el método send() para obtener el siguiente número primo.

Podemos hacer uso del método send() para interactuar con el generador y obtener el siguiente número primo cada vez que se envíe un valor.

In [21]:
def is_prime(num):
    """Función auxiliar para verificar si un número es primo."""
    if num <= 1:
        return False
    if num <= 3:
        return True
    if num % 2 == 0 or num % 3 == 0:
        return False
    i = 5
    while i * i <= num:
        if num % i == 0 or num % (i + 2) == 0:
            return False
        i += 6
    return True

def prime_coroutine():
    """Generador que produce números primos secuencialmente."""
    num = 2
    while True:
        if is_prime(num):
            received = yield num
            if received is not None:
                num = received
        num += 1

# Código para interactuar con la corutina
generator = prime_coroutine()  # Crea la corutina
print(next(generator))  # Inicia la corutina y obtiene el primer número primo

# Obtener los siguientes números primos
print(generator.send(None))  # Continuar desde el último número primo
print(generator.send(None))  # Continuar desde el último número primo
print(generator.send(None))  # Continuar desde el último número primo


2
3
5
7


#### Ejercicio 20: Programación funcional con map y reduce para procesamiento distribuido

Utiliza las funciones map y reduce de la biblioteca functools para encontrar la suma de los cuadrados de una lista de números distribuida entre varios procesos.



In [22]:
import multiprocessing
from functools import reduce

def square(n):
    """Devuelve el cuadrado de un número."""
    return n * n

def add(x, y):
    """Suma dos números."""
    return x + y

def parallel_sum_of_squares(numbers):
    """Calcula la suma de los cuadrados de una lista de números distribuidos entre varios procesos."""
    pool_size = multiprocessing.cpu_count()  # Número de procesos igual al número de núcleos disponibles
    with multiprocessing.Pool(processes=pool_size) as pool:
        # Distribuir el cálculo del cuadrado de cada número
        squares = pool.map(square, numbers)
        # Reducir la lista de cuadrados sumándolos
        result = reduce(add, squares)
    return result

# Lista de números
numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

# Ejecutar la función y mostrar el resultado
result = parallel_sum_of_squares(numbers)
print("La suma de los cuadrados es:", result)


La suma de los cuadrados es: 385
